In [1]:
!nvidia-smi

Tue May 11 14:49:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   35C    P8    10W / 280W |     62MiB / 11175MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 20%   

In [2]:
import os, time
import h5py
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
import tensorflow.keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import load_model

#K.set_image_dim_ordering('th')
tensorflow.keras.backend.set_image_data_format('channels_first')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
      # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


In [4]:
# path to the model files.
model_save_path = '/mnt/sdb1/zhuweilun/FASNet/weights/160_epoch30_msu.h5'

# Train

In [5]:
from tensorflow.keras.applications.vgg16 import VGG16

# dimensions of images. (less than 224x 224)
img_width, img_height = (160,160)

# nuumber of layers to freeze
nFreeze = 11

## MSU ##
train_data_dir = '/mnt/sdb1/zhuweilun/FASNet/msu_mfsd/train/'
nb_train_samples = 8664
nb_epoch = 30

def get_tr_vgg_model(img_width, img_height):
    
    # build the VGG16 network
    model = VGG16(weights='imagenet', include_top=False, input_shape = (3,img_width,img_height))
    
    #freeze layers
    for layer in model.layers[:nFreeze]:
        layer.trainable = False
        
    #model.summary()
    #print(len(model.layers))
    
    return model

def add_top_layers(model):
    
    combine_model = Sequential()
    combine_model.add(model)
    
    top_model = Sequential()
    top_model.add(Flatten(input_shape=model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    #top_model.add(Dense(2, activation='softmax'))
    top_model.add(Dense(1, activation='sigmoid'))

    # add the model on top of the convolutional base
    combine_model.add(top_model)
    
    return combine_model

def run_train(model):
    
    start_time = time.time()

    # compile model
    model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6),
#                   optimizer=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6),
#                 optimizer=optimizers.Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=1e-6),
#                 optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0),
#                   optimizer=optimizers.Adamax(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6),
              metrics=['accuracy'])
    
    print('Model Compiled.')
    
    train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip = True,
            fill_mode='nearest',
            data_format="channels_first")


    train_generator = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=(img_height, img_width),
            batch_size=200,
            class_mode='binary')
    
    label_map1 = (train_generator.class_indices)
    print(label_map1)


    print('\nFine-tuning top layers...\n')

#     earlyStopping = callbacks.EarlyStopping(monitor='val_loss',
#                                            patience=10, 
#                                            verbose=0, mode='min', restore_best_weights=True)

    #fit model
    #model.fit_generator(
    #       train_generator,
    #       callbacks=[earlyStopping],
    #      use_multiprocessing = False,
    #       steps_per_epoch=nb_train_samples/100,
    #      epochs=nb_epoch,
    #       validation_data=validation_generator,
    #       validation_steps=nb_validation_samples/100)
    
#     model.fit(
#            train_generator,
#            callbacks=[earlyStopping],
#           use_multiprocessing = False,
#            steps_per_epoch=nb_train_samples/200,
#           epochs=nb_epoch,
#            validation_data=validation_generator,
#            validation_steps=nb_validation_samples/200)

    model.fit(
            train_generator,
            use_multiprocessing = False,
            steps_per_epoch=nb_train_samples/200,
            epochs=nb_epoch)


    #model.save_weights(top_model_weights_path)
    model.save(model_save_path)
    
    print('\nDone fine-tuning, have a nice day!')
    print("\nExecution time %s seconds" % (time.time() - start_time))

In [6]:
if __name__ == "__main__":
   
    vgg16_tr_model = get_tr_vgg_model(img_width, img_height)
    vgg16_tr_model = add_top_layers(vgg16_tr_model)
    vgg16_tr_model.summary()
    
    # fine-tuning the model 
    run_train(vgg16_tr_model)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512, 5, 5)         14714688  
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 3277313   
Total params: 17,992,001
Trainable params: 16,256,513
Non-trainable params: 1,735,488
_________________________________________________________________
Model Compiled.
Found 8664 images belonging to 2 classes.
{'attack': 0, 'real': 1}

Fine-tuning top layers...

Epoch 1/30
 8/43 [====>.........................] - ETA: 23s - loss: 0.7213 - accuracy: 0.6381

UnknownError: 2 root error(s) found.
  (0) Unknown:  OSError: unrecognized data stream contents when reading image file
Traceback (most recent call last):

  File "/home/zhuweilun/.conda/envs/tf2/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 243, in __call__
    ret = func(*args)

  File "/home/zhuweilun/.conda/envs/tf2/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py", line 309, in wrapper
    return func(*args, **kwargs)

  File "/home/zhuweilun/.conda/envs/tf2/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 785, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/zhuweilun/.conda/envs/tf2/lib/python3.6/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 801, in wrapped_generator
    for data in generator_fn():

  File "/home/zhuweilun/.conda/envs/tf2/lib/python3.6/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 932, in generator_fn